# Client demo

Demonstrate the use of the Client (Assetto Corsa Sim interface) without the OpenAI Gym interface layer.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import sys
import os
import numpy as np
import pandas as pd
import glob as glob
import time
import pickle
from omegaconf import OmegaConf

# add custom paths
sys.path.extend([os.path.abspath('./assetto_corsa_gym')])
import AssettoCorsaEnv.assettoCorsa as assettoCorsa

# Configure the logging system
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO,  # Set the logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Format of the log messages
    datefmt='%Y-%m-%d %H:%M:%S',  # Format of the timestamp
)


pygame 2.6.0 (SDL 2.28.4, Python 3.9.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


# load config file

In [5]:
config = OmegaConf.load("config.yml")
client = assettoCorsa.make_client_only(config.AssettoCorsa)

2024-07-30 13:16:56 - AssettoCorsaEnv.ac_client - WARNING - Controls will be executed locally and not by the server


# Show static info

In [9]:
static_info = client.simulation_management.get_static_info()
ac_mod_config = client.simulation_management.get_config()

logger.info("Static info:")
for i in static_info:
    logger.info(f"{i}: {static_info[i]}")
logger.info("AC Mod config:")
for i in ac_mod_config:
    logger.info(f"{i}: {ac_mod_config[i]}")


2024-07-30 13:22:58 - __main__ - INFO - Static info:
2024-07-30 13:22:58 - __main__ - INFO - autoShifterOn: 1
2024-07-30 13:22:58 - __main__ - INFO - isCarInPitlane: 1
2024-07-30 13:22:58 - __main__ - INFO - TrackName: monza
2024-07-30 13:22:58 - __main__ - INFO - TrackLength: 5758.66064453125
2024-07-30 13:22:58 - __main__ - INFO - LastSplits: [0, 0, 0]
2024-07-30 13:22:58 - __main__ - INFO - CAR_WHEELBASE: 2.2663625188168828
2024-07-30 13:22:58 - __main__ - INFO - TrackConfiguration: 
2024-07-30 13:22:58 - __main__ - INFO - isCarInPit: 1
2024-07-30 13:22:58 - __main__ - INFO - WindSpeed: 21
2024-07-30 13:22:58 - __main__ - INFO - CAR_TRACK: 1.426605066284887
2024-07-30 13:22:58 - __main__ - INFO - penaltiesEnabled: 0
2024-07-30 13:22:58 - __main__ - INFO - CarName: ks_mazda_miata
2024-07-30 13:22:58 - __main__ - INFO - WindDirection: 359
2024-07-30 13:22:58 - __main__ - INFO - CAR_WHEEL_R: [0.2888000011444092, 0.2888000011444092, 0.2888000011444092, 0.2888000011444092]
2024-07-30 13:

# Apply actions and recover the car

In [10]:
states = []

client.reset()

for i in range(100):
    if i % 2 == 0:
        steer = .1
    else:
        steer = -.1
    client.controls.set_controls(steer=steer, acc=0.5, brake=-1.)
    client.respond_to_server()
    state = client.step_sim()
    states.append(state.copy())
    time.sleep(0.01)

client.controls.set_defaults()
client.respond_to_server()
client.simulation_management.send_reset()
client.close()

2024-07-30 13:28:05 - AssettoCorsaEnv.ac_client - INFO - sending reset to simulation management server
2024-07-30 13:28:06 - AssettoCorsaEnv.ac_client - INFO - AC Client. Listening at host: localhost port: 2345
2024-07-30 13:28:06 - AssettoCorsaEnv.ac_client - INFO - Client connected on 2345


KeyboardInterrupt: 

# Plot states

In [7]:
df = pd.DataFrame(states)
df.columns

Index(['done', 'LapInvalidated', 'actualGear', 'packetId', 'wheel_speed_rr',
       'tyres_load', 'cgHeight', 'rl_tire_temperature_core',
       'fr_tire_pressure', 'yaw', 'LastFF', 'completedLaps', 'fl_wheel_load',
       'tyre_slip_ratio_fr', 'rl_damper_linear_potentiometer', 'penaltyTime',
       'tyre_slip_ratio_rr', 'accelY', 'accelX', 'world_position_x',
       'world_position_y', 'world_position_z', 'wheel_speed_rl',
       'SlipAngle_fr', 'BestLap', 'Dy_fr', 'wheel_speed_fr',
       'local_velocity_z', 'local_velocity_y', 'local_velocity_x',
       'rr_tire_pressure', 'RPM', 'NdSlip', 'Dy_fl', 'wheel_speed_fl',
       'angular_velocity_x', 'angular_velocity_y', 'angular_velocity_z',
       'SlipAngle_fl', 'fl_damper_linear_potentiometer', 'velocity_x',
       'SuspensionTravel', 'Mz', 'tyre_slip_ratio_rl',
       'rr_tire_temperature_core', 'Dy_rl', 'velocity_z', 'rr_wheel_load',
       'pitch', 'tyre_slip_ratio_fl', 'timestamp_ac', 'currentTime',
       'LapCount', 'Dy_rr', 'i

In [ ]:
plt.title("Speed")
plt.plot(df.speed)
plt.show()

plt.title("steerAngle")
plt.plot(df.steerAngle)
plt.show()